In [1]:
import pandas as pd
df = pd.read_csv("imdb_movies.csv")

In [2]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10178 entries, 0 to 10177
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   names       10178 non-null  object 
 1   date_x      10178 non-null  object 
 2   score       10178 non-null  float64
 3   genre       10093 non-null  object 
 4   overview    10178 non-null  object 
 5   crew        10122 non-null  object 
 6   orig_title  10178 non-null  object 
 7   status      10178 non-null  object 
 8   orig_lang   10178 non-null  object 
 9   budget_x    10178 non-null  float64
 10  revenue     10178 non-null  float64
 11  country     10178 non-null  object 
dtypes: float64(3), object(9)
memory usage: 954.3+ KB


,score,budget_x,revenue
count,10178.000000,1.017800e+04,1.017800e+04
mean,63.497052,6.488238e+07,2.531401e+08
std,13.537012,5.707565e+07,2.777880e+08
min,0.000000,1.000000e+00,0.000000e+00
25%,59.000000,1.500000e+07,2.858898e+07
50%,65.000000,5.000000e+07,1.529349e+08
75%,71.000000,1.050000e+08,4.178021e+08
max,100.000000,4.600000e+08,2.923706e+09


In [3]:
df = df.rename(columns={
    'names': 'movie_title',
    'date_x': 'release_date',
    'score': 'imdb_rating',
    'budget_x': 'budget'
})

In [4]:
df.isnull().sum()

movie_title      0
release_date     0
imdb_rating      0
genre           85
overview         0
crew            56
orig_title       0
status           0
orig_lang        0
budget           0
revenue          0
country          0
dtype: int64

In [5]:
df = df.dropna(subset=['genre'])

In [6]:
df = df[(df['budget'] > 0) & (df['revenue'] > 0)]

In [7]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [8]:
df = df.dropna(subset=['release_year'])
df['release_year'] = df['release_year'].astype(int)

In [9]:
df['genre'] = df['genre'].str.split(', ')
df = df.explode('genre')

In [10]:
df = df.drop(columns=[
    'overview',
    'crew',
    'orig_title',
    'status',
    'orig_lang'
])

In [11]:
df['revenue'].describe()

count    1.002000e+04
mean     2.539591e+08
std      2.768803e+08
min      1.000000e+00
25%      2.934812e+07
50%      1.537162e+08
75%      4.200076e+08
max      2.923706e+09
Name: revenue, dtype: float64

In [12]:
upper_limit = df['revenue'].quantile(0.99)
df = df[df['revenue'] <= upper_limit]

In [13]:
df['profit'] = df['revenue'] - df['budget']

In [14]:
df['rating_category'] = pd.cut(
    df['imdb_rating'],
    bins=[0, 6, 7.5, 10],
    labels=['Low', 'Medium', 'High']
)

In [15]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 9919 entries, 0 to 10177
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   movie_title      9919 non-null   object        
 1   release_date     9919 non-null   datetime64[ns]
 2   imdb_rating      9919 non-null   float64       
 3   genre            9919 non-null   object        
 4   budget           9919 non-null   float64       
 5   revenue          9919 non-null   float64       
 6   country          9919 non-null   object        
 7   release_year     9919 non-null   int64         
 8   profit           9919 non-null   float64       
 9   rating_category  23 non-null     category      
dtypes: category(1), datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 784.7+ KB


,movie_title,release_date,imdb_rating,genre,budget,revenue,country,release_year,profit,rating_category
0,Creed III,2023-03-02,73.0,"Drama, Action",75000000.0,271616668.0,AU,2023,196616668.0,NaN
2,The Super Mario Bros. Movie,2023-04-05,76.0,"Animation, Adventure, Family, Fantasy, Comedy",100000000.0,724459031.0,AU,2023,624459031.0,NaN
3,Mummies,2023-01-05,70.0,"Animation, Comedy, Family, Adventure, Fantasy",12300000.0,34200000.0,AU,2023,21900000.0,NaN
4,Supercell,2023-03-17,61.0,Action,77000000.0,340941958.6,US,2023,263941958.6,NaN
5,Cocaine Bear,2023-02-23,66.0,"Thriller, Comedy, Crime",35000000.0,80000000.0,AU,2023,45000000.0,NaN


In [16]:
df['imdb_rating'].describe()

count    9919.000000
mean       63.667406
std        12.830168
min         0.000000
25%        59.000000
50%        65.000000
75%        71.000000
max        98.000000
Name: imdb_rating, dtype: float64

In [17]:
df['imdb_rating'] = df['imdb_rating'] / 10

In [18]:
df['imdb_rating'].describe()

count    9919.000000
mean        6.366741
std         1.283017
min         0.000000
25%         5.900000
50%         6.500000
75%         7.100000
max         9.800000
Name: imdb_rating, dtype: float64

In [19]:
df[df['imdb_rating'] == 0].shape

(169, 10)

In [20]:
df = df[df['imdb_rating'] > 0]

In [21]:
df['imdb_rating'].describe()

count    9750.000000
mean        6.477097
std         0.979695
min         1.000000
25%         5.900000
50%         6.600000
75%         7.100000
max         9.800000
Name: imdb_rating, dtype: float64

In [22]:
df['rating_category'] = pd.cut(
    df['imdb_rating'],
    bins=[1, 6, 7.5, 10],
    labels=['Low', 'Medium', 'High']
)

In [23]:
df.to_csv("imdb_movies_cleaned.csv", index=False)

In [24]:
df.head()

,movie_title,release_date,imdb_rating,genre,budget,revenue,country,release_year,profit,rating_category
0,Creed III,2023-03-02,7.3,"Drama, Action",75000000.0,271616668.0,AU,2023,196616668.0,Medium
2,The Super Mario Bros. Movie,2023-04-05,7.6,"Animation, Adventure, Family, Fantasy, Comedy",100000000.0,724459031.0,AU,2023,624459031.0,High
3,Mummies,2023-01-05,7.0,"Animation, Comedy, Family, Adventure, Fantasy",12300000.0,34200000.0,AU,2023,21900000.0,Medium
4,Supercell,2023-03-17,6.1,Action,77000000.0,340941958.6,US,2023,263941958.6,Medium
5,Cocaine Bear,2023-02-23,6.6,"Thriller, Comedy, Crime",35000000.0,80000000.0,AU,2023,45000000.0,Medium


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9750 entries, 0 to 10177
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   movie_title      9750 non-null   object        
 1   release_date     9750 non-null   datetime64[ns]
 2   imdb_rating      9750 non-null   float64       
 3   genre            9750 non-null   object        
 4   budget           9750 non-null   float64       
 5   revenue          9750 non-null   float64       
 6   country          9750 non-null   object        
 7   release_year     9750 non-null   int64         
 8   profit           9750 non-null   float64       
 9   rating_category  9727 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 771.4+ KB


In [26]:
df['rating_category'] = pd.cut(
    df['imdb_rating'],
    bins=[0.9, 6, 7.5, 10.1],
    labels=['Low', 'Medium', 'High']
)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9750 entries, 0 to 10177
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   movie_title      9750 non-null   object        
 1   release_date     9750 non-null   datetime64[ns]
 2   imdb_rating      9750 non-null   float64       
 3   genre            9750 non-null   object        
 4   budget           9750 non-null   float64       
 5   revenue          9750 non-null   float64       
 6   country          9750 non-null   object        
 7   release_year     9750 non-null   int64         
 8   profit           9750 non-null   float64       
 9   rating_category  9750 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 771.4+ KB


In [28]:
df.to_csv("imdb_movies_cleaned.csv", index=False)